In [1]:
# SETUP
#
# Install keras R
# install.packages('keras')
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

In [2]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params("inference")

Loading required package: rjson


In [3]:
# py_config()
print(paste0("OS: ", Sys.info()["sysname"]))
print(R.version$version.string)
print(paste0("Keras: ", packageVersion("keras")))
print(paste0("Tensorflow: ", packageVersion("tensorflow")))
print(paste0("Keras using ", backend()$backend()))
print(paste0("Keras channel ordering is ", backend()$image_data_format())) 

[1] "OS: Linux"
[1] "R version 3.4.1 (2017-06-30)"
[1] "Keras: 2.1.4"
[1] "Tensorflow: 1.5"
[1] "Keras using tensorflow"
[1] "Keras channel ordering is channels_last"


In [10]:
fake_data <- give_fake_data(batches = params$BATCH_SIZE * params$BATCHES_GPU, col_major = FALSE)

In [68]:
cat('x_train shape:', dim(fake_data), '\n')

x_train shape: 1280 224 224 3 


In [69]:
vec_list <- split(fake_data, as.factor(rep(1:params$BATCHES_GPU, each = params$BATCH_SIZE)))

batch_list <- lapply(vec_list, function(x) array(x, c(params$BATCH_SIZE, 224, 224, 3)))

In [74]:
# instantiate the model
model <- application_resnet50(include_top = FALSE, input_shape = c(224L, 224L, 3L))

In [14]:
#summary(model)

In [70]:
predict_on_batch_list <- function(batch, classifier) {predict_on_batch(classifier, batch)}

In [72]:
system.time(
    features <- lapply(batch_list, predict_on_batch_list, model)
)

   user  system elapsed 
 16.408   2.338  18.178 

In [82]:
system.time(
    features <- predict(model, x = fake_data, batch_size = params$BATCH_SIZE)
)

   user  system elapsed 
 15.178   2.336  16.816 

In [ ]:
# Return features from classifier
predict_fn <- function(classifier, data, batchsize){
    out <- array(0, dim = c(dim(data)[1], params$RESNET_FEATURES))
    idx <- 0:(dim(data)[1] %/% batchsize - 1)
    for (i in idx){
        dta <- data[(i*batchsize + 1):((i+1)*batchsize),,,]
        out[(i*batchsize + 1):((i+1)*batchsize), ] <- predict_on_batch(classifier, dta)
    }
    return(out)
}

In [16]:
# Time execution 
t <- system.time({
    features_cl = predict_fn(model, fake_data, params$BATCH_SIZE)
})
print(t)

   user  system elapsed 
 19.638   2.469  21.680 


In [17]:
# Time execution on channels-first data
print(paste("Images per second ", (params$BATCH_SIZE * params$BATCHES_GPU)/t[3]))

[1] "Images per second  59.040590405904"
